# 提取 - 使用聊天模型和少量示例从文本和其他非结构化媒体中提取结构化数据。

### 构建一个信息提取链

我们将使用聊天模型的 **工具调用（tool-calling）** 功能，从非结构化文本中提取结构化信息。我们还将演示如何在这种场景下使用 **少样本提示（few-shot prompting）** 来提升模型的表现效果。

### 模式（Schema）

首先，我们需要描述我们希望从文本中提取哪些信息。

我们将使用 Pydantic 来定义一个示例模式，用于提取个人信息。

In [1]:
from typing import Optional

from pydantic import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    # ^ Doc-string for the entity Person.
    # This doc-string is sent to the LLM as the description of the schema Person,
    # and it can help to improve extraction results.

    # Note that:
    # 1. Each field is an `optional` -- this allows the model to decline to extract it!
    # 2. Each field has a `description` -- this description is used by the LLM.
    # Having a good description can help improve extraction results.
    name: Optional[str] = Field(default=None, description="该人的姓名")
    hair_color: Optional[str] = Field(
        default=None, description="如果知道的话，请告知头发的颜色"
    )
    height_in_meters: Optional[str] = Field(
        default=None, description="高度以米为单位测量"
    )

在定义模式（schema）时，有两个最佳实践：

1. **对属性和整个模式进行详细描述**：这些信息会被发送给大语言模型（LLM），有助于提升信息提取的准确性。
2. **不要强制模型编造信息！** 如上所示，我们使用了 `Optional` 来允许模型在不知道答案时返回 `None`，而不是强行输出不存在的信息。

🔴 **重要提示**  
为了获得最佳效果，请对模式进行清晰的文档说明，并确保在文本中没有相关信息时，不要强制模型返回结果。

### 信息提取器（The Extractor）

让我们使用上面定义的模式，创建一个信息提取器。

In [2]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        # 你是一个专业信息提取算法。
        # 只需从文本中提取相关信息。
        # 如果你不知道某个需要提取属性的值，
        # 请将该属性的值返回为 null。
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value.",
        ),
        # Please see the how-to about improving performance with
        # reference examples.
        # MessagesPlaceholder('examples'),
        ("human", "{text}"),
    ]
)

In [3]:
import getpass
import os

try:
    # load environment variables from .env file (requires `python-dotenv`)
    from dotenv import load_dotenv

    _ = load_dotenv()
except ImportError:
    pass

if not os.environ.get("DASHSCOPE_API_KEY"):
  os.environ["DASHSCOPE_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")



In [11]:
from langchain_community.chat_models.tongyi import ChatTongyi

llm = ChatTongyi(
    streaming=True,
    name="qwen-vl-max"
)

In [12]:
structured_llm = llm.with_structured_output(schema=Person)

In [13]:
text = "七龙珠中拉姿丽·艾玛尔，性别女，身高大约165厘米，金发，种族是改造人"
prompt = prompt_template.invoke({"text": text})
structured_llm.invoke(prompt)

Person(name='拉姿丽·艾玛尔', hair_color='金发', height_in_meters='1.65')

### 多个实体（Multiple Entities）

在大多数情况下，你可能需要提取的是一**组实体**（entity list），而不是单个实体。

使用 Pydantic，你可以通过将模型相互嵌套的方式来轻松实现这一点。

In [14]:
from typing import List, Optional

from pydantic import BaseModel, Field


class Person(BaseModel):
    """Information about a person."""

    # 这是实体 "Person" 的文档字符串（doc-string）。
    # 这个文档字符串会被发送给大语言模型（LLM），作为对 "Person" 模型的描述，
    # 它有助于提升信息提取的准确性。
    
    # 注意：
    # 1. 每个字段都是 `Optional` 类型 —— 这样允许模型在无法识别时选择不提取该字段！
    # 2. 每个字段都有一个 `description` 描述 —— LLM 会使用这些描述来理解字段含义。
    # 提供清晰准确的描述可以显著提升提取效果。
    name: str = Field(...,description="该人的姓名")
    hair_color: Optional[str] = Field(
        default=None, description="如果知道的话，请告知头发的颜色"
    )
    height_in_meters: Optional[str] = Field(
        default=None, description="高度以米为单位测量"
    )


class Data(BaseModel):
    """Extracted data about people."""

    # Creates a model so that we can extract multiple entities.
    people: List[Person]

In [15]:
structured_llm = llm.with_structured_output(schema=Data)
text = "七龙珠中拉姿丽·艾玛尔，性别女，身高大约165厘米，金发，种族是改造人。还有孙悟空是主角，写别男"
prompt = prompt_template.invoke({"text": text})
res = structured_llm.invoke(prompt)
res

Data(people=[])

### 参考示例（Reference Examples）

可以通过**少样本提示**（few-shot prompting）来引导大语言模型（LLM）应用的行为。对于聊天模型来说，这可以表现为一系列**输入与响应消息的配对**，用以展示期望的行为。

例如，我们可以通过交替的用户和助手消息来传达某个符号的含义：

In [16]:
messages = [
    {"role": "user", "content": "2 🦜 2"},
    {"role": "assistant", "content": "4"},
    {"role": "user", "content": "2 🦜 3"},
    {"role": "assistant", "content": "5"},
    {"role": "user", "content": "3 🦜 4"},
]

response = llm.invoke(messages)
print(response.content)

7


结构化输出通常在底层使用 **工具调用（tool calling）** 机制。这通常包括生成包含工具调用的 AI 消息，以及包含工具调用结果的工具消息。

在这种情况下，一组消息的顺序应该是什么样的？

不同的聊天模型提供商对有效的消息序列有不同的要求。有些接受如下形式的（可重复的）消息序列：

- 用户消息  
- 包含工具调用的 AI 消息  
- 包含工具调用结果的工具消息  

而另一些则还要求一个最终的 AI 消息，其中包含某种形式的响应。

LangChain 提供了一个实用函数 `tool_example_to_messages`，它可以为大多数模型提供商生成一个合法的消息序列。它通过只需要提供相应的 Pydantic 工具调用表示，简化了结构化少样本示例的生成过程。

我们来试一下这个方法。我们可以将输入字符串和期望的 Pydantic 对象配对，转换为可以提供给聊天模型的一组消息。在底层，LangChain 会将这些工具调用格式化为各个模型提供商所需的格式。

> 注意：此版本的 `tool_example_to_messages` 要求 `langchain-core>=0.3.20`。

In [17]:
from langchain_core.utils.function_calling import tool_example_to_messages

examples = [
    (
        "海洋广阔而蔚蓝，深度超过 20,000 英尺",
        Data(people=[]),
    ),
    (
        "张小三从法国远道而来，前往西班牙。",
        Data(people=[Person(name="张小三", height_in_meters=None, hair_color=None)]),
    ),
]


messages = []

for txt, tool_call in examples:
    if tool_call.people:
        # This final message is optional for some providers
        ai_response = "检测到的人."
    else:
        ai_response = "未检测到人员."
    messages.extend(tool_example_to_messages(txt, [tool_call], ai_response=ai_response))

C:\Users\asus\AppData\Local\Temp\ipykernel_21340\4279074080.py:23: LangChainBetaWarning: The function `tool_example_to_messages` is in beta. It is actively being worked on, so the API may change.
  messages.extend(tool_example_to_messages(txt, [tool_call], ai_response=ai_response))


检查结果，我们看到这两个示例对生成了八条消息：

In [19]:
for message in messages:
    message.pretty_print()

================================ Human Message =================================

海洋广阔而蔚蓝，深度超过 20,000 英尺
================================== Ai Message ==================================
Tool Calls:
  Data (e4732f0f-66c6-47eb-8087-59d8ddf330f6)
 Call ID: e4732f0f-66c6-47eb-8087-59d8ddf330f6
  Args:
    people: []
================================= Tool Message =================================

You have correctly called this tool.
================================== Ai Message ==================================

未检测到人员.
================================ Human Message =================================

张小三从法国远道而来，前往西班牙。
================================== Ai Message ==================================
Tool Calls:
  Data (c10bbcee-fa81-4362-af90-f72df8d93c7e)
 Call ID: c10bbcee-fa81-4362-af90-f72df8d93c7e
  Args:
    people: [{'name': '张小三', 'hair_color': None, 'height_in_meters': None}]
================================= Tool Message =================================

You have correctly called